# Clasificadores

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.colors import ListedColormap

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [9]:
train = pd.read_csv ('/content/data_inlf_train.csv')

In [10]:
test = pd.read_csv ('/content/data_inlf_test.csv')

In [11]:
y = train['inlf']

In [12]:
x = train.drop(columns= 'inlf')

In [13]:
x.shape

(564, 8)

In [14]:
y.shape

(564,)

In [15]:
sc = StandardScaler()
sc.fit(x)
X_train_std = sc.transform(x)
X_test_std = sc.transform(test)

In [16]:
lr = LogisticRegression(C=100, solver='lbfgs', multi_class='ovr')
lr.fit(X_train_std, y)

LogisticRegression(C=100, multi_class='ovr')

In [17]:
y_pred = lr.predict(X_test_std)

In [18]:
lr.predict(X_test_std)

array([0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0])

In [19]:
Competencia = pd.DataFrame(data= y_pred, columns = ['Expected']). reset_index().rename(columns = {'index': 'id'})

In [20]:
Competencia. to_csv ('competencia.csv', index =False)

knn


In [21]:
from sklearn.experimental import enable_halving_search_cv
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [22]:
knn = KNeighborsClassifier(n_neighbors=5, p=2,
                          metric='minkowski')
knn.fit(X_train_std, y)

KNeighborsClassifier()

In [23]:
y_pred = knn.predict(test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


In [24]:
Competencia = pd.DataFrame(data= y_pred, columns = ['Expected']). reset_index().rename(columns = {'index': 'id'})

In [25]:
Competencia. to_csv ('competencia.csv', index =False)

In [26]:
pipe_svc = make_pipeline(StandardScaler(), KNeighborsClassifier())

In [27]:
param_grid = {'kneighborsclassifier__n_neighbors': [500],
             "kneighborsclassifier__metric":["euclidean","manhattan","minkowski"] }
rs = RandomizedSearchCV(estimator=pipe_svc,
      param_distributions=param_grid,
      scoring='accuracy',
      refit=True,
      n_iter=20,
      cv=10,
      random_state=1,
      n_jobs=-1)

In [28]:
rs.fit(x,y)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 3 is smaller than n_iter=20. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('standardscaler',
                                              StandardScaler()),
                                             ('kneighborsclassifier',
                                              KNeighborsClassifier())]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'kneighborsclassifier__metric': ['euclidean',
                                                                         'manhattan',
                                                                         'minkowski'],
                                        'kneighborsclassifier__n_neighbors': [500]},
                   random_state=1, scoring='accuracy')

In [29]:
y_pred = rs.predict(test)

In [30]:
competencia = pd.DataFrame(data= y_pred, columns = ['Expected']). reset_index().rename(columns = {'index': 'id'})

In [31]:
competencia. to_csv ('competencia.csv', index =False)

Param range

In [32]:
pipe_svc = make_pipeline(StandardScaler(), SVC(random_state=1))

In [33]:
param_range = [10000]

In [34]:
param_grid = [{'svc__C': param_range,
      'svc__kernel': ['linear']},
      {'svc__C': param_range,
      'svc__gamma': param_range,
      'svc__kernel': ['rbf']}]

In [35]:
hs = HalvingRandomSearchCV(pipe_svc,
    param_distributions=param_grid,
    n_candidates='exhaust',
    resource='n_samples',
    factor=1.5,
    random_state=1,
    n_jobs=-1)

In [36]:
hs.fit(x,y)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 2 is smaller than n_iter=28. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


HalvingRandomSearchCV(estimator=Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('svc', SVC(random_state=1))]),
                      factor=1.5, n_jobs=-1,
                      param_distributions=[{'svc__C': [10000],
                                            'svc__kernel': ['linear']},
                                           {'svc__C': [10000],
                                            'svc__gamma': [10000],
                                            'svc__kernel': ['rbf']}],
                      random_state=1)

In [37]:
hs.predict(test)

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [38]:
y_pred = hs.predict(test)

In [39]:
competencia = pd.DataFrame(data= y_pred, columns = ['Expected']). reset_index().rename(columns = {'index': 'id'})

In [40]:
competencia. to_csv ('competencia.csv', index =False)